## **Spatio-Temporal Beam-Level Traffic Forecasting Challenge by ITU**
The objective of this challenge is to develop a multivariate time series forecasting model for traffic volume (DLThpVol) at the beam level.
* 2880 beams across 30 base stations
* each base station consists of 3 cells with 32 beams , with data recorded hourly
* dataset encompasses a five week period with data recorded at hourly intervals

* datasets:
    * traffic_DLThpVol.csv: represents throughput volume.
    * traffic_DLThpTime.csv: represents throughput time.
    * traffic_ DLPRB.csv: represents Physical Resource Block (PRB) utilization.
    * traffic_MR_number.csv: represents user count.

### **Notebook Objective**
* Feature Engineering:

### Note
* For this to work you need a **high ram** environment. The normal 12gb ram from colab will get maxed out easily but the high ram from google colab pro is sufficient enough
* Also this code takes a while to run: OVER 1 HR

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import random
import gc
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
import torch
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = 500
pd.options.display.max_rows = 600

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
 #cpu vars
    torch.manual_seed(seed_value)
# cpu  vars
    random.seed(seed_value)
 # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
# gpu vars
        torch.backends.cudnn.deterministic = True
 #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

random_seed(42,True)

### **Train and Test Preparation**

In [ ]:
path = '/content/gdrive/MyDrive/spatio_temporal/prepared_data/'
test = pd.read_csv(path + "spatio_temporal_test.csv")
test['daily_hr'] = test['hr'] % 24
test['day'] = test['hr']//24
train = pd.read_parquet(path + 'transformed_dlthtpvol.parquet')
train['ID'] = "traffic_DLThpVol_train_"+ train['week'].astype('str') + '_' + train['hr'].astype('str') + '_' + train['base_station'].astype('str') + '_' +train['cell_type'].astype('str') + "_"+  train['beam'].astype('str')
train['daily_hr'] = train['hr']%24
train['day'] = train['hr']//24


display(test.head(), train.head())

,ID,Target,week,hr,base_station,cell_type,beam,daily_hr,day
0,traffic_DLThpVol_test_5w-6w_0_0_0_0,0,6,0,0,0,0,0,0
1,traffic_DLThpVol_test_5w-6w_1_0_0_0,0,6,1,0,0,0,1,0
2,traffic_DLThpVol_test_5w-6w_2_0_0_0,0,6,2,0,0,0,2,0
3,traffic_DLThpVol_test_5w-6w_3_0_0_0,0,6,3,0,0,0,3,0
4,traffic_DLThpVol_test_5w-6w_4_0_0_0,0,6,4,0,0,0,4,0


,ID,Target,week,hr,base_station,cell_type,beam,daily_hr,day
0,traffic_DLThpVol_train_1_0_0_0_0,0.000,1,0,0,0,0,0,0
1,traffic_DLThpVol_train_1_0_0_0_1,0.000,1,0,0,0,1,0,0
2,traffic_DLThpVol_train_1_0_0_0_2,0.000,1,0,0,0,2,0,0
3,traffic_DLThpVol_train_1_0_0_0_3,0.000,1,0,0,0,3,0,0
4,traffic_DLThpVol_train_1_0_0_0_4,0.031,1,0,0,0,4,0,0


### **Additional Datasets**

In [ ]:
time = pd.read_parquet(path + "transformed_dlthptime.parquet").rename(columns = {'Target':'time'})
prb = pd.read_parquet(path + "transformed_dlprb.parquet").rename(columns = {'Target': 'prb'})
mrno = pd.read_parquet(path + "transformed_mrnumber.parquet").rename(columns = {'Target': 'mrno'})
display(time.head(), prb.head(), mrno.head())

,ID,time,week,hr,base_station,cell_type,beam
0,traffic_DLThpVol_test_1w_0_0_0,0.000000,1,0,0,0,0
1,traffic_DLThpVol_test_1w_0_0_1,0.040340,1,0,0,0,1
2,traffic_DLThpVol_test_1w_0_0_2,0.006534,1,0,0,0,2
3,traffic_DLThpVol_test_1w_0_0_3,0.000000,1,0,0,0,3
4,traffic_DLThpVol_test_1w_0_0_4,0.323395,1,0,0,0,4


,ID,prb,week,hr,base_station,cell_type,beam
0,traffic_DLThpVol_test_1w_0_0_0,0.066698,1,0,0,0,0
1,traffic_DLThpVol_test_1w_0_0_1,0.085882,1,0,0,0,1
2,traffic_DLThpVol_test_1w_0_0_2,0.000000,1,0,0,0,2
3,traffic_DLThpVol_test_1w_0_0_3,0.090813,1,0,0,0,3
4,traffic_DLThpVol_test_1w_0_0_4,0.214019,1,0,0,0,4


,ID,mrno,week,hr,base_station,cell_type,beam
0,traffic_DLThpVol_test_1w_0_0_0,0.000000,1,0,0,0,0
1,traffic_DLThpVol_test_1w_0_0_1,0.000000,1,0,0,0,1
2,traffic_DLThpVol_test_1w_0_0_2,0.000000,1,0,0,0,2
3,traffic_DLThpVol_test_1w_0_0_3,0.027642,1,0,0,0,3
4,traffic_DLThpVol_test_1w_0_0_4,0.055997,1,0,0,0,4


In [ ]:
train = pd.merge(train, time[['base_station', 'beam', 'cell_type', 'hr', 'week', 'time']], on =['base_station', 'beam', 'cell_type', 'hr', 'week'], how ='left')
train = pd.merge(train, prb[['base_station', 'beam', 'cell_type', 'hr', 'week', 'prb']], on =['base_station', 'beam', 'cell_type', 'hr', 'week'], how ='left')
train = pd.merge(train, mrno[['base_station', 'beam', 'cell_type', 'hr', 'week', 'mrno']], on =['base_station', 'beam', 'cell_type', 'hr', 'week'], how ='left')
del time
del prb
del mrno
train.head()

,ID,Target,week,hr,base_station,cell_type,beam,daily_hr,day,time,prb,mrno
0,traffic_DLThpVol_train_1_0_0_0_0,0.000,1,0,0,0,0,0,0,0.000000,0.066698,0.000000
1,traffic_DLThpVol_train_1_0_0_0_1,0.000,1,0,0,0,1,0,0,0.040340,0.085882,0.000000
2,traffic_DLThpVol_train_1_0_0_0_2,0.000,1,0,0,0,2,0,0,0.006534,0.000000,0.000000
3,traffic_DLThpVol_train_1_0_0_0_3,0.000,1,0,0,0,3,0,0,0.000000,0.090813,0.027642
4,traffic_DLThpVol_train_1_0_0_0_4,0.031,1,0,0,0,4,0,0,0.323395,0.214019,0.055997


In [ ]:
print(train['ID'].duplicated().sum())

0


In [ ]:
print(train['week'].unique())
print(test['week'].unique())

[1 2 3 4 5]
[ 6 11]


In [ ]:
# sns.distplot(train['Target'])

In [ ]:
# sns.distplot(np.log1p(train['Target']))

In [ ]:
# sns.distplot(np.sqrt(train['Target']))

### **Cross Validation**

In [ ]:
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)
# Create folds
train['fold'] = -1
for fold, (_, val_idx) in enumerate(skf.split(X=train, y=train['base_station'])):
    train.loc[val_idx, 'fold'] = fold

In [ ]:
train['fold'].value_counts()

,count
fold,
5,241920
3,241920
1,241920
7,241920
6,241920
4,241920
0,241920
9,241920
8,241920


### **Feature Engineering Workflow**

This feature engineering pipeline is designed to generate rolling, expanding, and lagged features for time-series data related to base stations, cells, and beams, improving the dataset for machine learning models.

1. **Rolling Features**:
    - The `create_rolling_features` function generates rolling statistics like mean, median, standard deviation, and quantiles over specified windows for each group of `base_station`, `beam`, and `cell_type`.
    - These rolling statistics are calculated after shifting the data by a `shift_period` (e.g., 168 hours which represents a full week) to prevent data leakage from future observations.

2. **Expanding Features**:
    - The `apply_expanding_combinations` function computes expanding statistics (like mean and standard deviation) across various shifts for different combinations of group columns.
    - These features capture long-term trends across shifts of 168, 336, and 504 hours.

3. **Weekly Mean Features**:
    - The weekly average of the `Target` column is calculated and used to create a feature that tracks the previous week’s mean target.
    - This helps capture temporal trends at both the global and station/beam/cell level.

4. **Lagged Features**:
    - Lagged versions of `Target`, `mrno`, and `prb` are created for different time shifts. This helps the model understand how past values influence the current time step.
    - The lag is applied across `base_station`, `beam`, `cell_type`, and a combination of `daily_hr` and `day` to capture patterns recurring on a weekly basis.

5. **Final Dataset**:
    - The transformed features are split back into train and test sets based on their unique IDs, ready for model training.


In [ ]:
def create_rolling_features(data, group_cols, target_col, windows, shift_period, min_period, statistics):
    def apply_statistic(x, stat):
        rolled = x.shift(shift_period).rolling(window=window, min_periods=min_period)
        if stat == 'mean':
            return rolled.mean()
        elif stat == 'median':
            return rolled.median()
        elif stat == 'std':
            return rolled.std()
        elif stat.startswith('quantile_'):
            q = float(stat.split('_')[1])
            return rolled.quantile(q)
        else:
            raise ValueError(f"Unknown statistic: {stat}")

    for window in windows:
        for stat in statistics:
            stat_name = stat if not stat.startswith('quantile_') else f"{stat.split('_')[1]}th"
            col_name = f'rolling_previous_{target_col}_{stat_name}_{window}'

            data[col_name] = data.groupby(group_cols)[target_col].transform(
                lambda x: apply_statistic(x, stat)
            )

    return data
def apply_expanding_combinations(df, group_cols_list, target_col='Sales', shift_periods=[1], min_periods=1, stats=['mean', 'std']):
    # Loop through the group column combinations
    for group_cols in group_cols_list:
        # Generate base name for the grouping
        group_name = '_'.join(group_cols)

        for shift_period in shift_periods:
            for stat in stats:
                expanding_col_name = f'expanding_{group_name}_{target_col}_shift_{shift_period}_{stat}'

                # Apply groupby, shift, and expanding for the given statistic
                df[expanding_col_name] = df.groupby(group_cols)[target_col].transform(
                    lambda x: x.shift(shift_period).expanding(min_periods=min_periods).agg(stat)
                )

    return df




def feature_engineering(train, test):
    data = pd.concat([train, test])
    data.sort_values(by=['week', 'hr', 'base_station', 'cell_type', 'beam', 'daily_hr', 'day'], inplace=True)
    data = data.reset_index(drop=True)

    group_cols = ['base_station', 'beam', 'cell_type']
    windows = [336, 168]
    shift_period = 168
    min_period = 168
    statistics = ['mean', 'median', 'std', 'quantile_0.25', 'quantile_0.75']

    data = create_rolling_features(data, group_cols,'Target', windows, shift_period, min_period, statistics)
    windows = [7]
    shift_period = 168
    min_period = 7
    data = create_rolling_features(data, group_cols,'Target', windows, shift_period, min_period, statistics)

    group_cols = ['base_station', 'beam', 'cell_type']
    windows = [336, 168]
    shift_period = 168
    min_period = 168
    statistics = ['mean', 'median', 'std', 'quantile_0.25', 'quantile_0.75']

    data = create_rolling_features(data, group_cols,'prb', windows, shift_period, min_period, statistics)

    group_cols = ['base_station', 'beam', 'cell_type']
    windows = [336, 168]
    shift_period = 168
    min_period = 168
    statistics = ['mean', 'median', 'std', 'quantile_0.25', 'quantile_0.75']

    data = create_rolling_features(data, group_cols,'time', windows, shift_period, min_period, statistics)




    data = apply_expanding_combinations(
        data,
        [group_cols],
        target_col='Target',
        shift_periods=[168, 336, 504],
        min_periods=1,
        stats=['mean']
    )
    data = apply_expanding_combinations(
        data,
        [group_cols],
        target_col='prb',
        shift_periods=[168, 336, 504],
        min_periods=1,
        stats=['mean']
    )
    data = apply_expanding_combinations(
        data,
        [group_cols],
        target_col='mrno',
        shift_periods=[168, 336, 504],
        min_periods=1,
        stats=['mean']
    )
    data = apply_expanding_combinations(
        data,
        [group_cols],
        target_col='time',
        shift_periods=[168, 336, 504],
        min_periods=1,
        stats=['mean']
    )

    weekly_mean_dict = data.groupby('week')['Target'].mean().to_dict()
    data['previous_weekly_target_mean'] = data['week'].map(lambda x: weekly_mean_dict.get(x - 1))
    del weekly_mean_dict


    beam_celltype_weekly_dict = data.groupby(['base_station','beam', 'cell_type', 'week'])['Target'].mean().to_dict()
    data['previous_bs_beam_celltype_weekly_target_mean'] = data.apply(
        lambda row: beam_celltype_weekly_dict.get((row['base_station'], row['beam'], row['cell_type'], row['week'] - 1)), axis=1
    )
    del beam_celltype_weekly_dict


    for col in ['Target', 'mrno','prb']:
        for s in [1, 2, 3, 4]:
            data[f"{col}_shifted_b_b_c_dh_{s}"] = data.groupby(['base_station', 'beam', 'cell_type', 'daily_hr', 'day'])[col].shift(s) #same hour previous week



    train = data[data['ID'].isin(train['ID'].unique())]
    test = data[data['ID'].isin(test['ID'].unique())]

    return train, test

train_df, test_df = feature_engineering(train, test)


In [ ]:
print(test_df.shape)

(967680, 74)


In [ ]:
print(test_df.isnull().sum())

ID                                                                  0
Target                                                              0
week                                                                0
hr                                                                  0
base_station                                                        0
cell_type                                                           0
beam                                                                0
daily_hr                                                            0
day                                                                 0
time                                                           967680
prb                                                            967680
mrno                                                           967680
fold                                                           967680
rolling_previous_Target_mean_336                                    0
rolling_previous_Tar

In [ ]:
del train
del test

### **TARGET ENCODING**
This code is focused on applying **target encoding** to both training and test data, specifically for cross-validation, while minimizing data leakage between folds. Here's a breakdown of the key components:

#### 1. **Target Encoding**:
   - **Target encoding** replaces categorical variables (like `base_station` or `beam`) with aggregated statistics (mean, median, std, etc.) of the target variable (`Target`, `mrno`, `prb`) based on groupings. This process helps capture relationships between categorical features and the target.
   - The function **`target_encode`** generates target encodings by calculating various statistics (mean, skew, percentiles, etc.) for target columns, grouped by specified columns (like `base_station`, `beam`, etc.).

#### 2. **Cross-validation with Target Encoding**:
   - **Why use folds (cross-validation) for target encoding?**:
     - **Data leakage**: Without using folds, if you calculate encodings using the full dataset, the model might overfit because it will use information from the validation set during training. By calculating encodings separately for each fold, we ensure that each validation set does not use future information from its own targets.
   - The `perform_target_encoding` function does this by splitting the data into folds, encoding the training data using only the data outside the fold, and then applying those encodings to the validation fold. This process is repeated for each fold, and the validation sets are then combined.

#### 3. **Process Flow**:
   - For each group of columns (like `base_station`, `beam`):
     1. Target encodings are calculated for the entire training set and applied to the test data.
     2. For each fold:
        - The training data for that fold is used to compute target encodings.
        - The validation set is then merged with these encodings (without leaking information from that fold).
     3. All the encoded validation sets are combined to form the final encoded training set.

#### 4. **Why Target Encoding with Folds?**:
   - **Cross-validation consistency**: Using folds ensures that the model sees new target encoding values during validation that were calculated using only the training data. This method improves the robustness and generalization of the model by simulating real-world unseen data scenarios during training.
   - **Handling high cardinality**: Grouping columns like `base_station`, `beam`, and `cell_type` results in many unique combinations, which are effectively captured by target encoding and used as continuous features.


In [ ]:

def target_encode(train, target_cols, group_cols, stats=['mean']):
    encodings = {}

    for target_col in target_cols:
        target_encodings = {}
        for stat in stats:
            if stat == 'mean':
                encoding = train.groupby(group_cols)[target_col].mean().reset_index()
            elif stat == 'median':
                encoding = train.groupby(group_cols)[target_col].median().reset_index()
            elif stat == 'std':
                encoding = train.groupby(group_cols)[target_col].std().reset_index()
            elif stat == 'max':
                encoding = train.groupby(group_cols)[target_col].max().reset_index()
            elif stat == 'min':
                encoding = train.groupby(group_cols)[target_col].min().reset_index()
            elif stat == 'skew':
                encoding = train.groupby(group_cols)[target_col].apply(lambda x: skew(x)).reset_index()
            elif stat == 'kurtosis':
                encoding = train.groupby(group_cols)[target_col].apply(lambda x: kurtosis(x)).reset_index()
            elif stat in ['25th_percentile', '50th_percentile', '75th_percentile', '95th_percentile']:
                percentile = float(stat.split('_')[0].replace('th', '')) / 100
                encoding = train.groupby(group_cols)[target_col].quantile(percentile).reset_index()
            else:
                raise ValueError(f"Unsupported statistic: {stat}")

            encoding = encoding.rename(columns={target_col: f"{target_col}_{'_'.join(group_cols)}_{stat}_te"})
            target_encodings[stat] = encoding

        encodings[target_col] = target_encodings

    return encodings

# Initializing the OOF and fold predictions for both approaches
oof_preds_1 = np.zeros(len(train_df))
oof_preds_2 = np.zeros(len(train_df))
fold_pred_1 = []
fold_pred_2 = []

mae_approach_1 = []
mae_approach_2 = []

n_splits = train_df['fold'].nunique()
selected_columns =  ['daily_hr', 'hr', 'base_station', 'cell_type', 'beam']+ [col for col in train_df.columns if 'previous' in col] + [col for col in train_df.columns if 'shifted' in col]
target_col = 'Target'
target_cols = ['Target','mrno','prb']
group_cols_list = [
    ['base_station'],
    ['hr'],
    ['daily_hr'],
    ['beam'],

    ['base_station', 'beam'],
    ['base_station', 'hr'],
    ['base_station', 'daily_hr'],
    ['base_station', 'cell_type'],
    ['base_station', 'cell_type', 'daily_hr'],
    ['base_station', 'cell_type', 'beam' ],
    ['base_station', 'cell_type', 'beam',  'daily_hr'],
    ['base_station', 'cell_type','beam',  'day'],
    ['base_station', 'beam', 'daily_hr'],


    ['beam', 'daily_hr'],
    ['beam', 'cell_type'],
    ['beam', 'cell_type', 'daily_hr'],

]
stats_to_use = ['mean', 'std', 'skew', 'min', 'max', '25th_percentile', '75th_percentile','95th_percentile']
def perform_target_encoding(train_df, test_df, target_cols, group_cols_list, stats_to_use):
    n_splits = train_df['fold'].nunique()
    encoded_train_dfs = []

    # Encode test set using the entire training data
    for group_cols in group_cols_list:
        encodings = target_encode(train_df, target_cols, group_cols, stats_to_use)
        for target_col, target_encodings in encodings.items():
            for stat, enc in target_encodings.items():
                col_name = f"{target_col}_{'_'.join(group_cols)}_{stat}_te"
                if col_name not in test_df.columns:
                    test_df = test_df.merge(enc, on=group_cols, how='left')

    # Encode train set per fold
    for fold in range(n_splits):
        print(f"Performing target encoding for fold {fold+1}/{n_splits}")
        train_fold = train_df[train_df['fold'] != fold]
        val_fold = train_df[train_df['fold'] == fold]

        for group_cols in group_cols_list:
            encodings = target_encode(train_fold, target_cols, group_cols, stats_to_use)
            for target_col, target_encodings in encodings.items():
                for stat, enc in target_encodings.items():
                    col_name = f"{target_col}_{'_'.join(group_cols)}_{stat}_te"
                    if col_name not in val_fold.columns:
                        val_fold = val_fold.merge(enc, on=group_cols, how='left')

        encoded_train_dfs.append(val_fold)

        # Free memory
        del train_fold, val_fold
        gc.collect()

    # Concatenate all encoded validation sets
    encoded_train_df = pd.concat(encoded_train_dfs).sort_index()

    return encoded_train_df, test_df

encoded_train_df, encoded_test_df = perform_target_encoding(train_df, test_df, target_cols, group_cols_list, stats_to_use)

Performing target encoding for fold 1/10
Performing target encoding for fold 2/10
Performing target encoding for fold 3/10
Performing target encoding for fold 4/10
Performing target encoding for fold 5/10
Performing target encoding for fold 6/10
Performing target encoding for fold 7/10
Performing target encoding for fold 8/10
Performing target encoding for fold 9/10
Performing target encoding for fold 10/10


In [ ]:
encoded_train_df.to_parquet(path + 'encoded_train_final.parquet', index=False)
encoded_test_df.to_parquet(path + 'encoded_test_final.parquet', index=False)